# SAT formulation for Multiple Couriers planning

## 1. Direct constraint modelling

#### **Testing**: retrieve the first feasible solution

In [1]:
from z3 import *
import numpy as np
from itertools import combinations
from encodings_utils import *

# Modelling MCP problem
def mcp(instance, timeout=3000000):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of items
    l = instance["l"]  # Weights each courier can carry
    s = instance["s"]  # Items' sizes
    max_times = instance["max_times"]  # Time horizon: a courier can carry at most max_times items..
    # .. and since at each timestep a courier can pick exactly one item, max_times refers to the max time steps possible

    solver = Solver()
    solver.set("timeout", timeout)

    # VARIABLES
    # - Represent that each courier c picks the item i at the time step t (add 1 for depot)
    v = [[[Bool(f"x_{c}_{i}_{t}") for t in range(max_times + 1)] for i in range(n + 1)] for c in range(m)]
    
    # CONSTRAINTS
    # 1) Each courier c can carry at most l[c] kg
    for c in range(m):
        weight_set = []
        for i in range(n):
            for t in range(1, max_times):
                for _ in range(s[i]):
                    weight_set.append(v[c][i][t])
        solver.add(at_most_k_seq(weight_set, l[c], f"courier_{c}_load"))
        
    # 2) Each courier c starts and ends at position i = n
    for c in range(m):
        solver.add(v[c][n][0] == True)  # Start at position n at time 0
        solver.add(v[c][n][max_times] == True)  # End at position n at max_times
    
    # 3) Each courier must deliver at least one item
    for c in range(m):
        solver.add(at_least_one_bw([v[c][i][t] for t in range(1, max_times) for i in range(n)]))
    
    # 4) Each courier cannot pick the same item more than once
    for c in range(m):
        for i in range(n):
            solver.add(at_most_one_bw([v[c][i][t] for t in range(1, max_times)], f"exactly_once_courier{c}"))

    # 5) Each item is taken exactly once among all couriers and across all timesteps
    for i in range(n):
        solver.add(exactly_one_bw([v[c][i][t] for t in range(1, max_times) for c in range(m)], f"exactly_once_{i}"))
    
    # 6) All items should be picked up
    for i in range(n):
        solver.add(at_least_one_bw([v[c][i][t] for t in range(1, max_times) for c in range(m)]))

    '''
    # 7) Symmetry breaking: two couriers have the same load size
    for c1, c2 in combinations(range(m), 2):
        if l[c1] == l[c2]:
            for t in range(1, max_times):
                for i in range(n):
                    solver.add(Or(Not(v[c1][i][t]), v[c2][i][t]))
    '''

    # return model and the updated 3D variable
    return solver, v

# 6. Distance computation
def distance_constraint(instance, upperbound, v):
    m = instance["m"]  
    n = instance["n"]  
    max_times = instance["max_times"] 

    # Flattened list of Boolean variables representing distance usage
    all_distance_vars = []
    distance_vars = {}

    # Create distance variables and constraints
    for c in range(m):
        for t in range(max_times):
            for start in range(n + 1):
                for end in range(n + 1):
                    if start != end:
                        distance_var = Bool(f"dist_{c}_{start}_{end}_{t}") # type: ignore
                        all_distance_vars.append(distance_var)
                        distance_vars[(c, start, end, t)] = distance_var

                        # Define constraints for distance variables
                        if start < n and end < n:  # Ensure valid indices
                            travel_start = v[c][start][t]
                            travel_end = v[c][end][t + 1]
                            
                            # Encode: distance_var -> (travel_start and travel_end)
                            solver.add(Or(Not(distance_var), travel_start)) # type: ignore
                            solver.add(Or(Not(distance_var), travel_end)) # type: ignore
                            
                            # Encode: (travel_start and travel_end) -> distance_var
                            solver.add(Or(Not(travel_start), Not(travel_end), distance_var)) # type: ignore
 
    # Use at_most_k_seq to ensure the total distance does not exceed upper_bound
    solver.add(at_most_k_seq(all_distance_vars, upperbound, "distance_limit"))

def compute_distances(solution, instance):
    distances = instance["D"]
    n = instance["n"]
    distances_dict = {}

    for i, courier_solution in enumerate(solution):
        if not courier_solution:
            continue
        total_distance = 0
        depot = n+1  # Starting node
        for node in courier_solution:
            if node != depot:
                total_distance += distances[depot-1][node-1]
                depot = node
        total_distance += distances[depot-1][n]  # Return to starting node
        distances_dict[i] = total_distance

    return distances_dict

# Objective function
def compute_max_dist(distances_dict):
    return max(distances_dict.values())

# Print the total distance for each courier
def print_total_distances(solution, distances_dict):    
    for courier in distances_dict.keys():
        print(f"Courier {courier + 1} path: {solution[courier]}")
        print(f"Courier {courier + 1} total distance: {distances_dict[courier]}")

def extract_solution(model, instance):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of packages
    max_times = instance["max_times"]
    solution = []
    for c in range(m):
        courier_solution = []
        for t in range(max_times + 1):
            for i in range(n + 1):
                if model[v[c][i][t]]:
                    courier_solution.append(i + 1)  # Store 1-based index for better readability
        solution.append(courier_solution)
    return solution

# Example function to compute additional info for the instance
def compute_additional_info(instance):
    n = instance["n"]
    m = instance["m"]
    D = instance["D"]

    # Number of max_times steps
    max_times = (n // m) + 3

    # Minimum load (min among all sizes)
    min_load = min(instance["s"])

    # Max load (max among all sizes)
    max_load = max(instance["s"])

    # Maximum distance
    max_distance = np.sum(np.max(D, axis=1))

    additional_info = {
        "max_times": max_times,
        "max_dist": max_distance,
        "min_dist": 0,
        "min_load": min_load,
        "max_load": max_load,
    }

    return additional_info

# Importing instance
instance_num = "02"
file_path = os.path.join('Instances', f'inst{instance_num}.dat')
instance = read_dat_file(file_path)
instance.update(compute_additional_info(instance))

# Running the model
solver, v = mcp(instance=instance)

# add distance constraint
distance_constraint(instance, instance["max_dist"], v)

# Check satisfiability
if solver.check() == sat: # type: ignore
    model = solver.model()
    solution = extract_solution(model, instance)
    distances_dict = compute_distances(solution, instance)
    print_total_distances(solution, distances_dict)
    max_dist = compute_max_dist(distances_dict)
    print(f"\nObj. function value, max dist: {max_dist}")
else: print("unsat")

## Linear search

In [3]:
from z3 import * # type: ignore
import numpy as np # type: ignore
from encodings_utils import *

# Modelling MCP problem
def mcp(instance, timeout=3000000):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of items
    l = instance["l"]  # Weights each courier can carry
    s = instance["s"]  # Items' sizes
    max_times = instance["max_times"]  # Time horizon: a courier can carry at most max_times items..
    # .. and since at each timestep a courier can pick exactly one item, max_times refers to the max time steps possible

    solver = Solver() # type: ignore
    solver.set("timeout", timeout)

    # VARIABLES
    # - Represent that each courier c picks the item i at the time step t (add 1 for depot)
    v = [[[Bool(f"x_{c}_{i}_{t}") for t in range(max_times + 1)] for i in range(n + 1)] for c in range(m)] # type: ignore
    
    # CONSTRAINTS
    # 1) Each courier c can carry at most l[c] kg
    for c in range(m):
        weight_set = []
        for i in range(n):
            for t in range(1, max_times):
                for _ in range(s[i]):
                    weight_set.append(v[c][i][t])
        solver.add(at_most_k_seq(weight_set, l[c], f"courier_{c}_load"))
        
    # 2) Each courier c starts and ends at position i = n
    for c in range(m):
        solver.add(v[c][n][0] == True)  # Start at position n at time 0
        solver.add(v[c][n][max_times] == True)  # End at position n at max_times
    
    # 3) Each courier must deliver at least one item
    for c in range(m):
        solver.add(at_least_one_bw([v[c][i][t] for t in range(1, max_times) for i in range(n)]))
    
    # 4) Each courier cannot pick the same item more than once
    for c in range(m):
        for i in range(n):
            solver.add(at_most_one_bw([v[c][i][t] for t in range(1, max_times)], f"exactly_once_courier{c}"))

    # 5) Each item is taken exactly once among all couriers and across all timesteps
    for i in range(n):
        solver.add(exactly_one_bw([v[c][i][t] for t in range(1, max_times) for c in range(m)], f"exactly_once_{i}"))
    
    # 6) All items should be picked up
    for i in range(n):
        solver.add(at_least_one_bw([v[c][i][t] for t in range(1, max_times) for c in range(m)]))

    # Return model and the updated 3D variable
    return solver, v

# 6. The distance should be lower than a given upperbound
def distance_constraint(instance, solver, v, upperbound):
    m = instance["m"]  
    n = instance["n"]  
    max_times = instance["max_times"] 

    # Flattened list of Boolean variables representing distance usage
    all_distance_vars = []
    distance_vars = {}

    if upperbound <= 0:
        raise ValueError("Upper bound must be positive")

    # Create distance variables and constraints
    for c in range(m):
        for t in range(max_times):
            for start in range(n + 1):
                for end in range(n + 1):
                    if start != end:
                        distance_var = Bool(f"dist_{c}_{start}_{end}_{t}") # type: ignore
                        all_distance_vars.append(distance_var)
                        distance_vars[(c, start, end, t)] = distance_var

                        # Define constraints for distance variables
                        if start < n and end < n:  # Ensure valid indices
                            travel_start = v[c][start][t]
                            travel_end = v[c][end][t + 1]
                           
                            # Encode: distance_var -> (travel_start and travel_end)
                            solver.add(Or(Not(distance_var), travel_start)) # type: ignore
                            solver.add(Or(Not(distance_var), travel_end)) # type: ignore
                            
                            # Encode: (travel_start and travel_end) -> distance_var
                            solver.add(Or(Not(travel_start), Not(travel_end), distance_var)) # type: ignore
    
    # Use at_most_k_seq to ensure the total distance does not exceed upper_bound
    solver.add(at_most_k_seq(all_distance_vars, upperbound, "distance_limit"))

# Calculate distances for each courier
def compute_distances(solution, instance):
    distances = instance["D"]
    n = instance["n"]
    distances_dict = {}

    for i, courier_solution in enumerate(solution):
        if not courier_solution:
            continue
        total_distance = 0
        depot = n+1  # Starting node
        for node in courier_solution:
            if node != depot:
                total_distance += distances[depot-1][node-1]
                depot = node
        total_distance += distances[depot-1][n]  # Return to starting node
        distances_dict[i] = total_distance

    return distances_dict

# Objective function
def compute_max_dist(distances_dict):
    return max(distances_dict.values())

# Retrieve the model's solution
def extract_solution(model, instance, v):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of packages
    max_times = instance["max_times"]
    solution = []
    for c in range(m):
        courier_solution = []
        for t in range(max_times + 1):
            for i in range(n + 1):
                if model[v[c][i][t]]:
                    courier_solution.append(i + 1)  # Store 1-based index for better readability
        solution.append(courier_solution)
    return solution

# Example function to compute additional info for the instance
def compute_additional_info(instance):
    n = instance["n"]
    m = instance["m"]
    D = instance["D"]

    # Number of max_times steps
    max_times = (n // m) + 3

    # Minimum load (min among all sizes)
    min_load = min(instance["s"])

    # Max load (max among all sizes)
    max_load = max(instance["s"])

    # Maximum distance
    max_distance = np.sum(np.max(D, axis=1))

    additional_info = {
        "max_times": max_times,
        "max_dist": max_distance,
        "min_dist": 1,
        "min_load": min_load,
        "max_load": max_load,
    }

    return additional_info

# Linear search algo: starting from the bottom, since this is a minimization problem
def run_sat_linear_search(instance):
    
    # Initialize the Z3 solver and variables
    solver, v = mcp(instance)  # mcp returns the solver and variables

    # Set the original upper bound and lower bound for linear search
    original_upper_bound = instance["max_dist"]
    lower_bound = instance["min_dist"]
    upper_bound = original_upper_bound

    print("upperbound:", original_upper_bound)

    # Step size for the linear search 
    step = 5

    # Linear search for finding a feasible solution
    distance_limit = lower_bound
    best_solution = None
    best_objective_value = float('inf')

    # continue until the upperbound is not crossed
    while distance_limit <= upper_bound:

        # Save the current state of the solver
        solver.push()

        # Add the distance constraint to the solver
        distance_constraint(instance, solver, v, upperbound=distance_limit)

        # Check if the problem is satisfiable
        if solver.check() == sat:  # Solution found
            model = solver.model()
            solution = extract_solution(model, instance, v)
            distances_dict = compute_distances(solution, instance)
            objective_value = compute_max_dist(distances_dict)
            
            # Update the best solution if the current one is better
            if objective_value < best_objective_value:
                best_solution = solution
                best_objective_value = objective_value

            # No need to increase distance_limit further if this solution is optimal
            break

        # Increment the distance limit for the next iteration
        distance_limit += step

        # Restore the solver state for the next iteration
        solver.pop()

    # Return the best solution found and its objective value, or "unsat" if no feasible solution
    if best_solution is not None:
        return best_solution, best_objective_value
    else:
        return "unsat", None

# Importing instance
instance_num = "01"
file_path = os.path.join('Instances', f'inst{instance_num}.dat')
instance = read_dat_file(file_path)

# Compute additional info based on the instance
instance.update(compute_additional_info(instance))

# Run the linear search SAT solver
result, obj_value = run_sat_linear_search(instance)

# Process the final result
if result == "unsat":
    print("No solution found within the distance constraints.")
else:
    print("Best Solution Found:")
    for courier_idx, path in enumerate(result):
        print(f"Courier {courier_idx + 1}: {path}")

    print(f"Objective Function Value: {obj_value}")

upperbound: 44
Best Solution Found:
Courier 1: [7, 2, 4, 5, 6, 7]
Courier 2: [7, 1, 3, 7]
Objective Function Value: 16


## Binary search

In [4]:
from z3 import *
import numpy as np
from encodings_utils import *

# Modelling MCP problem
def mcp(instance, timeout=3000000):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of items
    l = instance["l"]  # Weights each courier can carry
    s = instance["s"]  # Items' sizes
    max_times = instance["max_times"]  # Time horizon

    solver = Solver()
    solver.set("timeout", timeout)

    # VARIABLES
    v = [[[Bool(f"x_{c}_{i}_{t}") for t in range(max_times + 1)] for i in range(n + 1)] for c in range(m)]
    
    # CONSTRAINTS
    for c in range(m):
        weight_set = []
        for i in range(n):
            for t in range(1, max_times):
                for _ in range(s[i]):
                    weight_set.append(v[c][i][t])
        solver.add(at_most_k_seq(weight_set, l[c], f"courier_{c}_load"))
        
    for c in range(m):
        solver.add(v[c][n][0] == True)
        solver.add(v[c][n][max_times] == True)
    
    for c in range(m):
        solver.add(at_least_one_bw([v[c][i][t] for t in range(1, max_times) for i in range(n)]))
    
    for c in range(m):
        for i in range(n):
            solver.add(at_most_one_bw([v[c][i][t] for t in range(1, max_times)], f"exactly_once_courier{c}"))

    for i in range(n):
        solver.add(exactly_one_bw([v[c][i][t] for t in range(1, max_times) for c in range(m)], f"exactly_once_{i}"))
    
    for i in range(n):
        solver.add(at_least_one_bw([v[c][i][t] for t in range(1, max_times) for c in range(m)]))

    return solver, v

# 6. The distance should be lower than a given upperbound
def distance_constraint(instance, solver, v, upperbound):
    m = instance["m"]  
    n = instance["n"]  
    max_times = instance["max_times"] 

    all_distance_vars = []
    distance_vars = {}

    if upperbound <= 0:
        raise ValueError("Upper bound must be positive")

    for c in range(m):
        for t in range(max_times):
            for start in range(n + 1):
                for end in range(n + 1):
                    if start != end:
                        distance_var = Bool(f"dist_{c}_{start}_{end}_{t}")
                        all_distance_vars.append(distance_var)
                        distance_vars[(c, start, end, t)] = distance_var

                        if start < n and end < n:
                            travel_start = v[c][start][t]
                            travel_end = v[c][end][t + 1]
                           
                            solver.add(Or(Not(distance_var), travel_start))
                            solver.add(Or(Not(distance_var), travel_end))
                            solver.add(Or(Not(travel_start), Not(travel_end), distance_var))
    
    solver.add(at_most_k_seq(all_distance_vars, upperbound, "distance_limit"))

def compute_distances(solution, instance):
    distances = instance["D"]
    n = instance["n"]
    distances_dict = {}

    for i, courier_solution in enumerate(solution):
        if not courier_solution:
            continue
        total_distance = 0
        depot = n + 1  # Starting node
        for node in courier_solution:
            if node != depot:
                total_distance += distances[depot - 1][node - 1]
                depot = node
        total_distance += distances[depot - 1][n]  # Return to starting node
        distances_dict[i] = total_distance

    return distances_dict

def compute_max_dist(distances_dict):
    return max(distances_dict.values())

def extract_solution(model, instance, v):
    m = instance["m"]
    n = instance["n"]
    max_times = instance["max_times"]
    solution = []
    for c in range(m):
        courier_solution = []
        for t in range(max_times + 1):
            for i in range(n + 1):
                if is_true(model[v[c][i][t]]):  # Properly check the model's interpretation of the variable
                    courier_solution.append(i + 1)
        solution.append(courier_solution)
    return solution

def compute_additional_info(instance):
    n = instance["n"]
    m = instance["m"]
    D = instance["D"]

    max_times = (n // m) + 3
    min_load = min(instance["s"])
    max_load = max(instance["s"])
    max_distance = np.sum(np.max(D, axis=1))

    additional_info = {
        "max_times": max_times,
        "max_dist": max_distance,
        "min_dist": 1,
        "min_load": min_load,
        "max_load": max_load,
    }

    return additional_info

def run_sat_binary_search(instance):
    solver, v = mcp(instance)
    original_upper_bound = instance["max_dist"]
    lower_bound = instance["min_dist"]
    upper_bound = original_upper_bound

    best_solution = None
    best_objective_value = float('inf')

    while lower_bound <= upper_bound:
        distance_limit = (lower_bound + upper_bound) // 2
        print(f"Trying distance limit: {distance_limit}")

        solver.push()
        distance_constraint(instance, solver, v, upperbound=distance_limit)

        if solver.check() == sat:
            model = solver.model()
            solution = extract_solution(model, instance, v)
            distances_dict = compute_distances(solution, instance)
            objective_value = compute_max_dist(distances_dict)
            
            if objective_value < best_objective_value:
                best_solution = solution
                best_objective_value = objective_value

            upper_bound = distance_limit - 1
        else:
            lower_bound = distance_limit + 1

        solver.pop()

    if best_solution is not None:
        return best_solution, best_objective_value
    else:
        return "unsat", None

# Importing instance
instance_num = "01"
file_path = os.path.join('Instances', f'inst{instance_num}.dat')
instance = read_dat_file(file_path)
instance.update(compute_additional_info(instance))

# Run the binary search SAT solver
result, obj_value = run_sat_binary_search(instance)

# Process the final result
if result == "unsat":
    print("No solution found within the distance constraints.")
else:
    print("Best Solution Found:")
    for courier_idx, path in enumerate(result):
        print(f"Courier {courier_idx + 1}: {path}")

    print(f"Objective Function Value: {obj_value}")

Trying distance limit: 22
Trying distance limit: 11
Trying distance limit: 5
Trying distance limit: 2
Trying distance limit: 1
Best Solution Found:
Courier 1: [7, 2, 4, 5, 6, 7]
Courier 2: [7, 1, 3, 7]
Objective Function Value: 16


## 2. Full adder-based modelling